# 5.2：时间演化块消减算法

Time-Evolving Block Decimation (TEBD)

> References:
> * _Efficient classical simulation of slightly entangled quantum computations_
> * _Efficient simulation of one-dimensional quantum many-body systems_
> * _Infinite time-evolving block decimation algorithm beyond unitary evolution_

步骤：
1. 随机初始化MPS：$|\varphi\rangle$；
2. 计算虚时长度为$\tau$的局域虚时演化算符：$e^{-\tau\hat{H}(p)}$；
3. 演化MPS并归一化：$e^{-\tau\hat{H}}|\varphi\rangle \rightarrow |\varphi\rangle/Z$（$Z = \langle\varphi|\varphi\rangle$）：
   - 若MPS虚拟维数超过阈值（截断维数），则利用中心正交形式实现最优裁剪；
4. 若$|\varphi\rangle$收敛：
   a. 若$\tau$足够小，则返回计算结果，计算结束。
   b. 否则，减小$\tau$并回到步骤2。

## 计算技巧

### 将二体演化门写成两个三阶张量的缩并，方便后续计算

![tebd_evo_gate](./images/tebd_evo_gate.png)

不分解直接演化会导致 MPS 张量个数发生改变（如下图）；考虑到程序在处理长程耦合的通用性，我们这里不选择这种方式


![tebd_evo_step](./images/tebd_evo_step.png)

对演化算符分解后方便处理长程相互作用：演化前后MPS长度不变；

演化会破坏对应张量的正交性：对于第$l_1$与第$l_2$个自旋上的局域演化门，

作用之后，$l_{\text{L}}$与$l_{\text{R}}$上及其之间的所有张量失去正交性

![tebd_evo_step_detail](./images/tebd_evo_step_detailed.png)

> 演化区域中，张量失去正交性且虚拟指标维数增大

对于上图的左边来说

![tebd_evo_step_left](./images/tebd_evo_step_left.png)

> 缩并之后把 `g` 和 `b` 维度合并在一起
>
> 表达式是 $A'_{as(gb)} = \sum_{s'} A_{as'b}L_{sgs'}$
>

对于上图的中间张量来说

![tebd_evo_step_middle](./images/tebd_evo_step_middle.png)

> 相当于是张量和 $I$ 做直积之后合并维度，也就是
>
> $A'_{(ga)s(g'b)} = A_{asb}I_{gg'}$
>

对于上图的右边来说

![tebd_evo_step_right](./images/tebd_evo_step_right.png)

> 缩并之后把 `g` 和 `a` 维度合并在一起，相当于是
>
> $A^{\prime}_{(ga)sb} = \sum_{s^{\prime}} A_{as^{\prime}b}R_{sgs^{\prime}}$

In [1]:
import torch
from einops import einsum, rearrange
from itertools import permutations
from time import time

In [2]:
tries = 1000

# try two different ways to calculate the direct product of I and a
# Surprisingly, the einsum way is faster than the way in which no calculation is needed
for dim in [2, 4, 8, 16, 32, 64, 128]:
    I = torch.eye(dim, dtype=torch.int32)
    a = torch.randn(3, 5, 7, dtype=torch.float32)
    start = time()
    for _ in range(tries):
        Ia_einsum = einsum(I, a, "g0 g1, a s b -> g0 a s g1 b")
        Ia_einsum = rearrange(Ia_einsum, "g0 a s g1 b -> (g0 a) s (g1 b)")
    end = time()
    einsum_time = end - start

    start = time()
    for _ in range(tries):
        a_ = a.unsqueeze(0).unsqueeze(0).repeat(dim, dim, 1, 1, 1)
        perms = permutations(range(dim), r=2)
        x, y = zip(*perms)
        x = torch.tensor(x)
        y = torch.tensor(y)
        a_[x, y] = 0.0
        a_[y, x] = 0.0
        Ia_no_calculation = rearrange(a_, "g0 g1 a s b -> (g0 a) s (g1 b)")
    end = time()
    no_calculation_time = end - start
    assert torch.allclose(Ia_no_calculation, Ia_einsum)

    print(
        f"dim={dim}, {einsum_time=}s, {no_calculation_time=}s, diff={no_calculation_time - einsum_time}s"
    )

dim=2, einsum_time=0.6932711601257324s, no_calculation_time=0.019344091415405273s, diff=-0.6739270687103271s
dim=4, einsum_time=0.010883092880249023s, no_calculation_time=0.021711111068725586s, diff=0.010828018188476562s
dim=8, einsum_time=0.014249086380004883s, no_calculation_time=0.12641692161560059s, diff=0.1121678352355957s
dim=16, einsum_time=0.02582097053527832s, no_calculation_time=0.17697620391845703s, diff=0.1511552333831787s
dim=32, einsum_time=0.058001041412353516s, no_calculation_time=0.46379804611206055s, diff=0.40579700469970703s
dim=64, einsum_time=0.12069201469421387s, no_calculation_time=1.3788259029388428s, diff=1.258133888244629s
dim=128, einsum_time=0.36752891540527344s, no_calculation_time=4.834371089935303s, diff=4.466842174530029s


将二体演化算符写成两个三阶张量的收缩，在这里，我们采取一种简单的分解方式，即

$G_{abcd} \stackrel{\text{permute}}{\longrightarrow} G_{abdc} \stackrel{\text{reshape (bd)}\rightarrow g}{\longrightarrow} (\text{gl})_{agc}$

$I_{bb'}I_{dd'} \stackrel{\otimes}{\longrightarrow} (gr)_{bb'dd'} \stackrel{\text{permute}}{\longrightarrow} (\text{gr})_{bb'd'd} \stackrel{\text{reshape (b'd')}\rightarrow g}{\longrightarrow} (\text{gr})_{bgd}$

也就是如下图

![tebd_op_to_tensors](./images/tebd_op_to_tensors.png)

In [3]:
op = torch.randn(2, 2, 2, 2, dtype=torch.complex64)
I = torch.eye(2, dtype=torch.int32)

gr = einsum(I, I, "b0 b1, d0 d1 -> b0 b1 d0 d1")
gr = rearrange(gr, "b0 b1 d0 d1 -> b0 (b1 d1) d0")

gl = rearrange(op, "a b c d -> a (b d) c")

product = einsum(gl, gr.to(gl.dtype), "a g c , b g d -> a b c d")

assert torch.allclose(product, op)

### 尽量少地移动正交中心；保持裁剪的最优性

策略要点：尽量少地移动正交中心；保持裁剪的最优性

1. 将正交中心以移动至$l_L$与$l_R$二者中距离当前正交中心最近的位置；
2. 演化后，从$l_L$或$l_R$出发，对整个演化区地张量做正交化变换（不裁剪）；
3. 当MPS处于正交中心形式后，通过移动中心的奇异值分解进行虚拟维数裁剪；
4. 正交化后的正交中心要尽可能地靠近下一次演化格点的位置。

> 例：
> 
> 演化前正交中心位于 $l_c = 4$，下一次的演化算符分别作用于自旋 1 与 2，再下一次的演化作用于自旋 5 与 6，则最优的中心正交化策略如下
>
> ![tebd_center_orthogonalization_example](./images/tebd_center_orthogonalization_example.png)
>
> 左图正交中心位置变化：4 → 2 → None → 1 → 2
>
> 2 → None 是演化步骤，1 → 2 是裁剪步骤
> 
>
> 下一步演化位置为5与6（若设再下一步演化位置为0和1），则对应的正交中心位置变化：
> 2 → 5 → None → 6 → 5
>
> 5 → None 是演化步骤，6 → 5 是裁剪步骤
>

强调：非中心正交形式下不裁剪，否则不能达到最优裁剪

In [4]:
# TODO： Algorithms/MPS_Algo.py/tebd